# Testing

In [1]:
from components.obs import LocalObs
obs = LocalObs(source=1, mode="dino", model_name="facebook/dinov2-with-registers-small", device="cuda")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
emb = obs.get_embedding()

In [5]:
emb

array([-1.84828043e+00, -7.04906702e-01, -1.66756189e+00, -1.40146005e+00,
       -3.34473228e+00,  1.07779706e+00, -2.23611760e+00, -2.16413450e+00,
       -1.94104815e+00, -6.49509370e-01, -3.90177399e-01,  2.69240737e-01,
        7.16435850e-01, -9.72258747e-01, -1.01016521e+00, -1.52731743e-02,
        8.80450726e-01, -2.26934806e-01,  4.70897138e-01,  1.53294187e-02,
        1.31477937e-01,  1.11559324e-01, -2.37365346e-02, -5.53531945e-01,
       -2.53506303e-01, -5.96102774e-01,  6.19350135e-01,  1.30673444e+00,
       -4.18306738e-01,  3.37119788e-01,  2.13403177e+00, -3.88634264e-01,
        5.12408689e-02, -1.98971987e+00,  4.66368020e-01,  1.61234844e+00,
        2.87377924e-01, -1.29031301e+00,  1.72767985e+00,  8.30308557e-01,
       -9.18930590e-01,  3.24865103e-01, -1.19877136e+00,  8.70823324e-01,
        1.15334582e+00, -1.62473202e+00, -8.95833433e-01, -2.87788367e+00,
        9.95146513e-01,  5.59764564e-01, -8.87720361e-02, -3.57399821e-01,
       -1.38330591e+00, -

In [9]:
import numpy as np

In [11]:
x=[]

In [5]:
import torch
#base = torch.arange(self.ptr, self.ptr + self.size,
#                    device=self.device) % self.size

#seq_idx = (base.unsqueeze(1) - self.offsets) % self.size

In [23]:
(torch.arange(9,9+10).unsqueeze(1)-torch.tensor([0,1,2]))%10

tensor([[9, 8, 7],
        [0, 9, 8],
        [1, 0, 9],
        [2, 1, 0],
        [3, 2, 1],
        [4, 3, 2],
        [5, 4, 3],
        [6, 5, 4],
        [7, 6, 5],
        [8, 7, 6]])

In [43]:
import torch
size     = 100
offsets  = torch.tensor([0,1,3,6,10,15,21])
ptr      = 99 # newest = 33, oldest = 34 because it in add(s) first overrides then make +1 to ptr
N        = size

# --- old (broken) logic
# base_old = torch.arange(ptr, ptr+size) % N
# seq_old  = (base_old.unsqueeze(1) - offsets) % N
# print("OLD:")
# for b, row in zip(base_old.tolist(), seq_old.tolist()):
#     print(f"{b}: {row}")

# --- new logic
idx      = torch.arange(N)
max_off  = offsets.max()
left     = (idx <  ptr) & (idx - max_off >= 0)
right    = (idx >= ptr) & (idx - max_off >= ptr)
mask     = left | right
bases    = idx[mask]
seq_new  = (bases.unsqueeze(1) - offsets) % N
print("\nFIXED:")
for b, row in zip(bases.tolist(), seq_new.tolist()):
    print(f"{b}: {row}")

'''
FIXED:
21: [21, 20, 18, 15, 11, 6, 0] it could be startes with 0 base because buffer breakpoint at 33 and 34
22: [22, 21, 19, 16, 12, 7, 1]
23: [23, 22, 20, 17, 13, 8, 2]
24: [24, 23, 21, 18, 14, 9, 3]
25: [25, 24, 22, 19, 15, 10, 4]
26: [26, 25, 23, 20, 16, 11, 5]
27: [27, 26, 24, 21, 17, 12, 6]
28: [28, 27, 25, 22, 18, 13, 7]
29: [29, 28, 26, 23, 19, 14, 8]
30: [30, 29, 27, 24, 20, 15, 9]
31: [31, 30, 28, 25, 21, 16, 10]
32: [32, 31, 29, 26, 22, 17, 11]
33: [33, 32, 30, 27, 23, 18, 12] # good job i am proud of you
55: [55, 54, 52, 49, 45, 40, 34] # same
56: [56, 55, 53, 50, 46, 41, 35]
57: [57, 56, 54, 51, 47, 42, 36]
58: [58, 57, 55, 52, 48, 43, 37]
59: [59, 58, 56, 53, 49, 44, 38]
60: [60, 59, 57, 54, 50, 45, 39]
61: [61, 60, 58, 55, 51, 46, 40]
62: [62, 61, 59, 56, 52, 47, 41]
63: [63, 62, 60, 57, 53, 48, 42]
64: [64, 63, 61, 58, 54, 49, 43]
65: [65, 64, 62, 59, 55, 50, 44]
66: [66, 65, 63, 60, 56, 51, 45]
67: [67, 66, 64, 61, 57, 52, 46]
68: [68, 67, 65, 62, 58, 53, 47]
69: [69, 68, 66, 63, 59, 54, 48]
70: [70, 69, 67, 64, 60, 55, 49]
71: [71, 70, 68, 65, 61, 56, 50]
72: [72, 71, 69, 66, 62, 57, 51]
73: [73, 72, 70, 67, 63, 58, 52]
74: [74, 73, 71, 68, 64, 59, 53]
75: [75, 74, 72, 69, 65, 60, 54]
76: [76, 75, 73, 70, 66, 61, 55]
77: [77, 76, 74, 71, 67, 62, 56]
78: [78, 77, 75, 72, 68, 63, 57]
79: [79, 78, 76, 73, 69, 64, 58]
80: [80, 79, 77, 74, 70, 65, 59]
81: [81, 80, 78, 75, 71, 66, 60]
82: [82, 81, 79, 76, 72, 67, 61]
83: [83, 82, 80, 77, 73, 68, 62]
84: [84, 83, 81, 78, 74, 69, 63]
85: [85, 84, 82, 79, 75, 70, 64]
86: [86, 85, 83, 80, 76, 71, 65]
87: [87, 86, 84, 81, 77, 72, 66]
88: [88, 87, 85, 82, 78, 73, 67]
89: [89, 88, 86, 83, 79, 74, 68]
90: [90, 89, 87, 84, 80, 75, 69]
91: [91, 90, 88, 85, 81, 76, 70]
92: [92, 91, 89, 86, 82, 77, 71]
93: [93, 92, 90, 87, 83, 78, 72]
94: [94, 93, 91, 88, 84, 79, 73]
95: [95, 94, 92, 89, 85, 80, 74]
96: [96, 95, 93, 90, 86, 81, 75]
97: [97, 96, 94, 91, 87, 82, 76]
98: [98, 97, 95, 92, 88, 83, 77]
99: [99, 98, 96, 93, 89, 84, 78]'''


FIXED:
21: [21, 20, 18, 15, 11, 6, 0]
22: [22, 21, 19, 16, 12, 7, 1]
23: [23, 22, 20, 17, 13, 8, 2]
24: [24, 23, 21, 18, 14, 9, 3]
25: [25, 24, 22, 19, 15, 10, 4]
26: [26, 25, 23, 20, 16, 11, 5]
27: [27, 26, 24, 21, 17, 12, 6]
28: [28, 27, 25, 22, 18, 13, 7]
29: [29, 28, 26, 23, 19, 14, 8]
30: [30, 29, 27, 24, 20, 15, 9]
31: [31, 30, 28, 25, 21, 16, 10]
32: [32, 31, 29, 26, 22, 17, 11]
33: [33, 32, 30, 27, 23, 18, 12]
34: [34, 33, 31, 28, 24, 19, 13]
35: [35, 34, 32, 29, 25, 20, 14]
36: [36, 35, 33, 30, 26, 21, 15]
37: [37, 36, 34, 31, 27, 22, 16]
38: [38, 37, 35, 32, 28, 23, 17]
39: [39, 38, 36, 33, 29, 24, 18]
40: [40, 39, 37, 34, 30, 25, 19]
41: [41, 40, 38, 35, 31, 26, 20]
42: [42, 41, 39, 36, 32, 27, 21]
43: [43, 42, 40, 37, 33, 28, 22]
44: [44, 43, 41, 38, 34, 29, 23]
45: [45, 44, 42, 39, 35, 30, 24]
46: [46, 45, 43, 40, 36, 31, 25]
47: [47, 46, 44, 41, 37, 32, 26]
48: [48, 47, 45, 42, 38, 33, 27]
49: [49, 48, 46, 43, 39, 34, 28]
50: [50, 49, 47, 44, 40, 35, 29]
51: [51, 50, 48,

'\nFIXED:\n21: [21, 20, 18, 15, 11, 6, 0] it could be startes with 0 base because buffer breakpoint at 33 and 34\n22: [22, 21, 19, 16, 12, 7, 1]\n23: [23, 22, 20, 17, 13, 8, 2]\n24: [24, 23, 21, 18, 14, 9, 3]\n25: [25, 24, 22, 19, 15, 10, 4]\n26: [26, 25, 23, 20, 16, 11, 5]\n27: [27, 26, 24, 21, 17, 12, 6]\n28: [28, 27, 25, 22, 18, 13, 7]\n29: [29, 28, 26, 23, 19, 14, 8]\n30: [30, 29, 27, 24, 20, 15, 9]\n31: [31, 30, 28, 25, 21, 16, 10]\n32: [32, 31, 29, 26, 22, 17, 11]\n33: [33, 32, 30, 27, 23, 18, 12] # good job i am proud of you\n55: [55, 54, 52, 49, 45, 40, 34] # same\n56: [56, 55, 53, 50, 46, 41, 35]\n57: [57, 56, 54, 51, 47, 42, 36]\n58: [58, 57, 55, 52, 48, 43, 37]\n59: [59, 58, 56, 53, 49, 44, 38]\n60: [60, 59, 57, 54, 50, 45, 39]\n61: [61, 60, 58, 55, 51, 46, 40]\n62: [62, 61, 59, 56, 52, 47, 41]\n63: [63, 62, 60, 57, 53, 48, 42]\n64: [64, 63, 61, 58, 54, 49, 43]\n65: [65, 64, 62, 59, 55, 50, 44]\n66: [66, 65, 63, 60, 56, 51, 45]\n67: [67, 66, 64, 61, 57, 52, 46]\n68: [68, 67,

In [245]:
def get_filtered_frames(frames: np.ndarray, steps: int = 70):
    lin_steps = np.round(np.linspace(1, 27, steps)).astype(int)
    indices = np.cumsum(np.insert(lin_steps, 0, 0))[:-1]
    indices = indices[indices < len(frames)]
    filtered_frames = frames[indices]
    # Convert from HWC to CHW format and create torch tensor
    if len(filtered_frames.shape) == 4:  # Batch of frames
        filtered_frames = np.transpose(filtered_frames, (0, 3, 1, 2))
    return filtered_frames
original=list(range(10000))
m=get_filtered_frames(np.arange()len(original)*-1-1
print([original[i] for i in m.tolist()]))

[9999, 9998, 9997, 9995, 9993, 9990, 9987, 9984, 9980, 9976, 9972, 9967, 9962, 9956, 9950, 9944, 9937, 9930, 9923, 9915, 9907, 9898, 9889, 9880, 9870, 9860, 9850, 9839, 9828, 9816, 9804, 9792, 9779, 9766, 9753, 9739, 9725, 9710, 9695, 9680, 9664, 9648, 9632, 9615, 9598, 9580, 9562, 9544, 9525, 9506, 9487, 9467, 9447, 9426, 9405, 9384, 9362, 9340, 9318, 9295, 9272, 9248, 9224, 9200, 9175, 9150, 9125, 9099, 9073, 9046]


In [1]:
from components.shrink import LogStencilMemory
import numpy as np

In [3]:
lgs=LogStencilMemory()

In [206]:
import socket
import argparse
import time
def send_udp_message(message, host='127.0.0.1', port=5005, timeout=1.0):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(timeout)
    try:
        sock.sendto(message.encode(), (host, port))
        response, _ = sock.recvfrom(1024)
        return response.decode()
    except socket.timeout:
        return "[Timeout] No response from server"
    finally:
        sock.close()


print(send_udp_message("GET"))
#print(send_udp_message("RESET"))
#time.sleep(1)
#print(send_udp_message("2"))  # Press right arrow key

0.000000


In [23]:
import sys
import time
import numpy as np
import torch

sys.path.append("components")  # optional: make sure components is importable

from env import SocketAppEnv

def test_env_run(max_steps=30):
    env = SocketAppEnv(max_steps=max_steps, device="cpu")

    obs, _ = env.reset()
    print("[Env Reset]")
    assert isinstance(obs, np.ndarray), "Observation must be numpy array"
    assert obs.shape == (env.state_dim,), f"Invalid observation shape: {obs.shape}"

    total_extrinsic = 0.0
    total_intrinsic = 0.0

    for step in range(max_steps):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)

        # Rough split: assume extrinsic comes from external UDP, IR is from tracker
        # Since we can't exactly decouple them, track deltas
        total_extrinsic += max(0.0, reward)  # non-negative assumption
        total_intrinsic += max(0.0, reward - total_extrinsic)

        print(f"[Step {step:02d}] Action={action} Reward={reward:.3f} Done={truncated}")

        if terminated or truncated:
            break

    print("\n=== Episode Summary ===")
    print(f"Total steps:     {step + 1}")
    print(f"Final reward:    {reward:.3f}")
    print(f"Total reward:    {total_extrinsic + total_intrinsic:.3f}")
    print(f"Env truncated:   {truncated}")

    env.close()
    print("[Env Closed]")

if __name__ == "__main__":
    test_env_run()


In [ ]:
import socket
def send_action_to_server(action_idx, host='127.0.0.1', port=5005):
    """Send a single action index to the UDP server."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        message = str(action_idx).encode()
        sock.sendto(message, (host, port))
        #print(f"[Sent] Action index {action_idx} to {host}:{port}")
    except Exception as e:
        print(f"[Error] {e}")
    finally:
        sock.close()
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as td
from components.nn import Actor, Q_Critic
from components.rollout import RolloutBufferNoDone, compute_gae
from components.ppo import ppo_update
#from components.action_module import send_action_async
from components.obs import LocalObs
from components.rew import E3BIntrinsicReward
from components.shrink import LogStencilMemory
from time import sleep
from threading import Thread
from pynput import keyboard
import asyncio
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir="runs/ppo_run")


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
STATE_DIM, ACTION_DIM, SEQ_LEN = 384, 7, 70

# GLOBAL PAUSE STATE
paused = False

def hotkey_listener():
    def on_press(key):
        global paused
        if key == keyboard.Key.f9:
            paused = not paused
            sleep(0.1)
            print(f"[Pause toggled] Paused = {paused}")
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

def main():
    global paused
    # Start hotkey listener thread
    Thread(target=hotkey_listener, daemon=True).start()

    # Components
    obs  = LocalObs(source=1, mode="dino", model_name="facebook/dinov2-with-registers-small", device=DEVICE)
    e3b  = E3BIntrinsicReward(latent_dim=STATE_DIM, decay=1, ridge=0.1, device=DEVICE)

    actor = Actor(state_dim=STATE_DIM, action_dim=ACTION_DIM).to(DEVICE)
    critic = Q_Critic(shared_lstm=actor.lstm, action_dim=ACTION_DIM).to(DEVICE)
    optim_actor = optim.Adam(actor.parameters(), lr=3e-4)
    optim_critic = optim.Adam(critic.parameters(), lr=3e-4)
    buffer = RolloutBufferNoDone(2048, STATE_DIM, ACTION_DIM, SEQ_LEN, "cpu")

    step_count = 0

    print("[Started in PAUSED mode] Press F9 to toggle")

    while True:
        if paused:
            continue

        emb_np = obs.get_embedding()
        memo.add(emb_np[None, :])
        
        seq = memo.shrinked()
        emb_seq = torch.from_numpy(seq).float().to(DEVICE)
        emb_seq = emb_seq.unsqueeze(0)
        #emb = emb_seq[-1]

        #print(f"[Shape Debug] emb_seq shape: {emb_seq.shape}")
        #print(f"[Shape Debug] emb shape: {emb.shape}")

        ir     = e3b.compute(emb_np)

        logits = actor(emb_seq)
        
        #print(f"[Shape Debug] logits shape: {logits.shape}")
        
        dist   = td.Categorical(logits=logits.squeeze(0))
        action = dist.sample()
        logp   = dist.log_prob(action)
        act_onehot = F.one_hot(action, ACTION_DIM).float()

        send_action_to_server(action.item())
        sleep(0.05)
        #print(f"[Shape Debug] act_onehot shape: {act_onehot.shape}")
        act_onehot=act_onehot.unsqueeze(0)
        extrinsic = 0.0
        total_r   = extrinsic + ir
        writer.add_scalar("Reward/Total", total_r, step_count)

        value     = critic(emb_seq, act_onehot).squeeze()

        #print(f"[Shape Debug] value shape: {value.shape}")
        
        buffer.add(emb_seq[0].detach().cpu(), act_onehot.detach().cpu(), total_r, value.detach().cpu(), logp.detach().cpu())
        #assert False, "debug step end"
        step_count += 1
        writer.add_scalar("Reward/Total", total_r, step_count)
        if buffer.ready() and step_count % 256 == 0:
            s, a, r, v, lp = buffer.get()
            returns, adv = compute_gae(r, v)
            ppo_update(actor, critic, optim_actor, optim_critic, s.detach(), a.detach(), lp.detach(), returns.detach(), adv.detach())
            print(f"[PPO Update] Step {step_count}")

if __name__ == "__main__":
    main()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
W0613 14:02:04.203000 21112 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


[Started in PAUSED mode] Press F9 to toggle
[Pause toggled] Paused = True
[Pause toggled] Paused = False
[Pause toggled] Paused = True
[Pause toggled] Paused = False
[PPO Update] Step 2048
[PPO Update] Step 2304
[PPO Update] Step 2560
[PPO Update] Step 2816
[PPO Update] Step 3072
[PPO Update] Step 3328
[PPO Update] Step 3584
[PPO Update] Step 3840
[PPO Update] Step 4096
[PPO Update] Step 4352
[PPO Update] Step 4608
[Pause toggled] Paused = True


In [10]:
mem_len=1000
step=70
import numpy as np
lin_steps = np.round(np.linspace(1, max(2, mem_len // (step//2)), step)).astype(int)
indices = np.cumsum(np.insert(lin_steps, 0, 0))[:-1]
indices = indices[indices < mem_len]
reversed_indices = mem_len - 1 - indices
reversed_indices = reversed_indices[::-1]

In [22]:
indices=np.array([  0,   1,   2,   4,   6,   9,  12,  15,  19,  23,  28,  33,  38,
        44,  50,  56,  63,  70,  78,  86,  94, 103, 112, 122, 132, 142,
       153, 164, 176, 188, 200, 213, 226, 240, 254, 268, 283, 298, 313,
       329, 345, 362, 379, 396, 414, 432, 451, 470, 489, 509, 529, 550,
       571, 592, 614, 636, 659, 682, 705, 729, 753, 777, 802, 827, 853,
       879, 905, 932, 959, 987])
self.states[self.ptr - indices]

array([  0,   1,   2,   4,   6,   9,  12,  15,  19,  23,  28,  33,  38,
        44,  50,  56,  63,  70,  78,  86,  94, 103, 112, 122, 132, 142,
       153, 164, 176, 188, 200, 213, 226, 240, 254, 268, 283, 298, 313,
       329, 345, 362, 379, 396, 414, 432, 451, 470, 489, 509, 529, 550,
       571, 592, 614, 636, 659, 682, 705, 729, 753, 777, 802, 827, 853,
       879, 905, 932, 959, 987])

In [46]:
import numpy as np

class LogStencilMemory:
    def __init__(self, max_len=1000, feature_dim=384, offsets=None):
        self.max_len = max_len
        self.feature_dim = feature_dim
        self.memory = np.zeros((0, feature_dim), dtype=np.float32)
        if offsets is None:
            # fallback: uniform dense offsets for demonstration
            self.offsets = np.arange(0, min(max_len, 70), dtype=int)
        else:
            self.offsets = np.array(offsets, dtype=int)

    def add(self, element: np.ndarray):
        if element.shape != (1, self.feature_dim):
            raise ValueError(f"Element must have shape (1, {self.feature_dim})")
        self.memory = np.concatenate([self.memory, element], axis=0)
        if len(self.memory) > self.max_len:
            self.memory = self.memory[-self.max_len:]

    def shrinked(self) -> np.ndarray:
        mem_len = len(self.memory)
        if mem_len == 0:
            return np.zeros((0, self.feature_dim), dtype=np.float32)
        # Only use offsets such that (offset+1) <= mem_len
        valid_offsets = self.offsets[self.offsets + 1 <= mem_len]
        idxs = -(valid_offsets + 1)
        return self.memory[idxs]

# Example usage:
if __name__ == "__main__":
    rollout_offsets = np.array([0, 1, 2, 4, 6, 9, 12], dtype=int)
    mem = LogStencilMemory(max_len=20, feature_dim=4, offsets=rollout_offsets)
    for i in range(15):
        x = np.full((1, 4), i, dtype=np.float32)
        mem.add(x)
        if i % 5 == 0:
            shrink = mem.shrinked()
            print(f"At step {i}, shrinked:\n", shrink)


At step 0, shrinked:
 [[0. 0. 0. 0.]]
At step 5, shrinked:
 [[5. 5. 5. 5.]
 [4. 4. 4. 4.]
 [3. 3. 3. 3.]
 [1. 1. 1. 1.]]
At step 10, shrinked:
 [[10. 10. 10. 10.]
 [ 9.  9.  9.  9.]
 [ 8.  8.  8.  8.]
 [ 6.  6.  6.  6.]
 [ 4.  4.  4.  4.]
 [ 1.  1.  1.  1.]]


In [41]:
import torch
import numpy as np
from components.rollout import RolloutBufferNoDone

def test_rollout_buffer_offsets():
    # Configuration
    size       = 10
    state_dim  = 2
    action_dim = 1
    device     = "cpu"

    # Create buffer
    buf = RolloutBufferNoDone(
        size=size,
        state_dim=state_dim,
        action_dim=action_dim,
        device=device
    )

    # Fill buffer with deterministic data:
    # state = [i, i*10], action = [i%2], reward = i, value = i*0.1, logprob = i*0.01
    for i in range(size):
        s  = torch.tensor([i, i*10],  dtype=torch.float, device=device)
        a  = torch.tensor([i % 2],     dtype=torch.float, device=device)
        r  = torch.tensor(i,           dtype=torch.float, device=device)
        v  = torch.tensor(i * 0.1,     dtype=torch.float, device=device)
        lp = torch.tensor(i * 0.01,    dtype=torch.float, device=device)
        buf.add(s, a, r, v, lp)

    assert buf.ready(), "Buffer should be full after 10 adds"

    # Retrieve sequences
    states_seq, actions, rewards, values, logprobs = buf.get()

    # Basic shape checks
    seq_len = buf.seq_len
    assert states_seq.shape == (size, seq_len, state_dim)
    assert actions.shape    == (size, action_dim)
    assert rewards.shape    == (size,)
    assert values.shape     == (size,)
    assert logprobs.shape   == (size,)

    print("Shapes are correct:")
    print("  states_seq:", states_seq.shape)
    print("  actions   :", actions.shape)
    print("  rewards   :", rewards.shape)
    print("  values    :", values.shape)
    print("  logprobs  :", logprobs.shape)

    # Verify that the first batch entry reconstructs the correct historical states
    offsets = buf.offsets.cpu().numpy()   # array of length seq_len
    base0   = 0                          # base index for first entry (after wrap)
    first_seq = states_seq[0].cpu().numpy()

    for k, off in enumerate(offsets):
        idx = (base0 - off) % size
        expected = np.array([idx, idx * 10], dtype=np.float32)
        actual   = first_seq[k]
        assert np.allclose(actual, expected), (
            f"Mismatch at seq position {k}: "
            f"got {actual}, expected {expected}"
        )

    print("First sequence row matches expected values based on offsets.")
    print("All tests passed!")

if __name__ == "__main__":
    test_rollout_buffer_offsets()


Shapes are correct:
  states_seq: torch.Size([10, 70, 2])
  actions   : torch.Size([10, 1])
  rewards   : torch.Size([10])
  values    : torch.Size([10])
  logprobs  : torch.Size([10])
First sequence row matches expected values based on offsets.
All tests passed!


In [111]:
import torch
import numpy as np

# Paste in your RolloutBufferNoDone here (omitted for brevity)...

# For this test we redefine it inline:
class RolloutBufferNoDone:
    def __init__(self, size, state_dim, action_dim,
                 device="cpu", padding_value=0.0):
        self.size     = size
        self.device   = device
        self.full     = False
        self.ptr      = 0
        self.pad_val  = padding_value

        self.state    = torch.zeros(size, state_dim, device=device)
        self.action   = torch.zeros(size, action_dim, device=device)
        self.reward   = torch.zeros(size, device=device)
        self.value    = torch.zeros(size, device=device)
        self.logprob  = torch.zeros(size, device=device)

        offsets = np.array([  0,   1,   2,   4,   6,   9,  12,  15,  19,  23,
                              28,  33,  38,  44,  50,  56,  63,  70,  78,  86,
                              94, 103, 112, 122, 132, 142, 153, 164, 176, 188,
                             200, 213, 226, 240, 254, 268, 283, 298, 313, 329,
                             345, 362, 379, 396, 414, 432, 451, 470, 489, 509,
                             529, 550, 571, 592, 614, 636, 659, 682, 705, 729,
                             753, 777, 802, 827, 853, 879, 905, 932, 959, 987],
                            dtype=np.int64)
        self.offsets  = torch.as_tensor(offsets, device=device)
        self.seq_len  = len(self.offsets)

    def add(self, s, a, r, v, lp):
        self.state[self.ptr]   = s
        self.action[self.ptr]  = a
        self.reward[self.ptr]  = r
        self.value[self.ptr]   = v
        self.logprob[self.ptr] = lp
        self.ptr  = (self.ptr + 1) % self.size
        self.full = self.full or self.ptr == 0

    def ready(self):
        return self.full

    def get(self):
        if not self.full:
            raise RuntimeError("Buffer not filled yet!")

        N       = self.size
        p       = self.ptr
        max_off = int(self.offsets.max().item())

        idx = torch.arange(N, device=self.device)
        mask = (idx < p) | (idx > p + max_off - 1)
        bases = idx[mask]

        ages     = (bases - p) % N
        order    = ages.argsort()
        bases    = bases[order]

        seq_idx    = (bases.unsqueeze(1) - self.offsets) % N
        states_seq = self.state[seq_idx]

        return (
            states_seq,
            self.action[bases],
            self.reward[bases],
            self.value[bases],
            self.logprob[bases],
        )

    def get_latest_state_seq(self, new_state: torch.Tensor):
        N     = self.size
        p     = self.ptr
        avail = N if self.full else p
        seq = []
        for off in self.offsets.tolist():
            if off == 0:
                seq.append(new_state)
            elif off <= avail - 1:
                idx = (p - off) % N
                seq.append(self.state[idx])
            else:
                pad = torch.full((self.state.shape[1],), self.pad_val, device=self.device)
                seq.append(pad)
        return torch.stack(seq, dim=0).unsqueeze(0)


if __name__ == "__main__":
    torch.set_printoptions(sci_mode=False, precision=1)

    # 1) Initialize and fill buffer with 1000 distinct states
    SIZE = 1000
    DIM  = 3
    buf = RolloutBufferNoDone(size=SIZE, state_dim=DIM, action_dim=1)

    for i in range(SIZE):
        state = torch.tensor([i, i+0.1, i+0.2])
        buf.add(state, torch.tensor([0.0]), torch.tensor(0.0), torch.tensor(0.0), torch.tensor(0.0))

    print("Buffer full:", buf.ready())

    # 2) Call get() and check number of sequences = SIZE - max_offset
    max_off = int(buf.offsets.max().item())
    states_seq, *_ = buf.get()
    expected = SIZE - max_off
    actual   = states_seq.size(0)
    print(f"Expected sequences: {expected}, got: {actual}")
    assert actual == expected

    # 3) Verify each sequence is strictly contiguous (no wrap)
    for batch_idx in range(actual):
        seq_idxs = ((buf.ptr + batch_idx + max_off) % SIZE
                    - buf.offsets).tolist()  # compute bases in chronological
        retrieved = states_seq[batch_idx, :, 0].tolist()
        expected_vals = [float(idx) for idx in seq_idxs]
        assert retrieved == expected_vals, f"Mismatch at batch {batch_idx}"
    print("All sequences contiguous and correct!")

    # 4) Test get_latest_state_seq
    new = torch.tensor([999.5, 999.6, 999.7])
    last_seq = buf.get_latest_state_seq(new)
    print("Latest sequence shape:", last_seq.shape)
    print("Latest sequence (first few vals):", last_seq[0, :5, 0].tolist())

    # 5) Check that the newest element appears at position 0 in offsets==0
    assert last_seq[0, 0, 0] == 999.5

    print("get_latest_state_seq works correctly!")


Buffer full: True
Expected sequences: 13, got: 13
All sequences contiguous and correct!
Latest sequence shape: torch.Size([1, 70, 3])
Latest sequence (first few vals): [999.5, 999.0, 998.0, 996.0, 994.0]
get_latest_state_seq works correctly!


In [127]:
states_seq[4]

tensor([[991.0000, 991.1000, 991.2000],
        [990.0000, 990.1000, 990.2000],
        [989.0000, 989.1000, 989.2000],
        [987.0000, 987.1000, 987.2000],
        [985.0000, 985.1000, 985.2000],
        [982.0000, 982.1000, 982.2000],
        [979.0000, 979.1000, 979.2000],
        [976.0000, 976.1000, 976.2000],
        [972.0000, 972.1000, 972.2000],
        [968.0000, 968.1000, 968.2000],
        [963.0000, 963.1000, 963.2000],
        [958.0000, 958.1000, 958.2000],
        [953.0000, 953.1000, 953.2000],
        [947.0000, 947.1000, 947.2000],
        [941.0000, 941.1000, 941.2000],
        [935.0000, 935.1000, 935.2000],
        [928.0000, 928.1000, 928.2000],
        [921.0000, 921.1000, 921.2000],
        [913.0000, 913.1000, 913.2000],
        [905.0000, 905.1000, 905.2000],
        [897.0000, 897.1000, 897.2000],
        [888.0000, 888.1000, 888.2000],
        [879.0000, 879.1000, 879.2000],
        [869.0000, 869.1000, 869.2000],
        [859.0000, 859.1000, 859.2000],


In [45]:
def get(self):
    if not self.full:
        raise RuntimeError("Buffer not filled yet!")

    max_offset = int(self.offsets.max().item())
    N = self.size
    p = self.ptr

    # Build bases in chronological order: 0 .. N-1
    idx = torch.arange(N, device=self.device)

    # Only allow sequences that do not go left of 0
    mask = (idx - max_offset >= 0)
    bases = idx[mask]

    seq_idx = (bases.unsqueeze(1) - self.offsets) % N
    states_seq = self.state[seq_idx]

    return (states_seq,
            self.action[bases],
            self.reward[bases],
            self.value[bases],
            self.logprob[bases])


In [59]:
import torch
size     = 100
offsets  = torch.tensor([0,1,3,6,10,15,21])
ptr      = 32                         # newest = 31, oldest = 32
N        = size

# --- old (broken) logic
# base_old = torch.arange(ptr, ptr+size) % N
# seq_old  = (base_old.unsqueeze(1) - offsets) % N
# print("OLD:")
# for b, row in zip(base_old.tolist(), seq_old.tolist()):
#     print(f"{b}: {row}")

# --- new logic

# emulate inside class
idx = torch.arange(N)
left  = (idx <  ptr) & (idx - offsets.max() >= 0)
right = (idx >= ptr) & (idx - offsets.max() >= ptr)
mask  = left | right
bases = idx[mask]                          # tensor([2,3,4,5,6,7,8,9])
chron  = (bases - ptr) % N                 # tensor([3,4,5,6,7,8,9,0])
bases  = bases[chron.argsort()]            # tensor([2,3,4,5,6,7,8,9])

print("chronological bases ->", bases.tolist())
seq_idx = (bases.unsqueeze(1) - offsets) % N
for b,row in zip(bases.tolist(), seq_idx.tolist()):
    print(f"base {b}: {row}")


chronological bases -> [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
base 53: [53, 52, 50, 47, 43, 38, 32]
base 54: [54, 53, 51, 48, 44, 39, 33]
base 55: [55, 54, 52, 49, 45, 40, 34]
base 56: [56, 55, 53, 50, 46, 41, 35]
base 57: [57, 56, 54, 51, 47, 42, 36]
base 58: [58, 57, 55, 52, 48, 43, 37]
base 59: [59, 58, 56, 53, 49, 44, 38]
base 60: [60, 59, 57, 54, 50, 45, 39]
base 61: [61, 60, 58, 55, 51, 46, 40]
base 62: [62, 61, 59, 56, 52, 47, 41]
base 63: [63, 62, 60, 57, 53, 48, 42]
base 64: [64, 63, 61, 58, 54, 49, 43]
base 65: [65, 64, 62, 59, 55, 50, 44]
base 66: [66, 65, 63, 60, 56, 51, 45]
base 67: [67, 66, 64, 61, 57, 52, 46]
base 68: [68, 67, 65, 62, 58, 53, 47]
base 69: [69, 68, 66, 63, 59, 54, 48]
base 70: [70, 69, 67, 64, 60, 55, 49]
base 71: [71, 70, 68, 65, 61, 56, 50]
base 72: [72, 71, 69, 

In [ ]:
import numpy as np

from sb3_contrib import RecurrentPPO

model = RecurrentPPO("MlpLstmPolicy", "CartPole-v2", verbose=1)
model.learn(50_000)

vec_env = model.get_env()
obs = vec_env.reset()
# Cell and hidden state of the LSTM
lstm_states = None
num_envs = 1
episode_starts = np.ones((num_envs,), dtype=bool)
while True:
    action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
    # Note: vectorized environment resets automatically
    obs, rewards, dones, info = vec_env.step(action)
    episode_starts = dones
    vec_env.render("human")

Using cuda device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21       |
|    ep_rew_mean     | 21       |
| time/              |          |
|    fps             | 250      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 18.1          |
|    ep_rew_mean          | 18.1          |
| time/                   |               |
|    fps                  | 164           |
|    iterations           | 2             |
|    time_elapsed         | 1             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 1.3177283e-05 |
|    clip_fraction        

C:\ProgramData\anaconda3\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [85]:
idx = torch.arange(size)
(idx < ptr) | (idx > (ptr + 21))

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [103]:
size     = 100
offsets  = torch.tensor([0,1,3,6,10,15,21])
ptr      = 32  # newest = 31, oldest = 32
N        = size

idx = torch.arange(N)
#mask = (idx < ptr) | (idx > (ptr + 21-1))
#bases = idx[mask]

# 1) Mask out any base index whose history would wrap the circular boundary
mask = (idx < ptr) | (idx > ptr + max_off-1)
bases = idx[mask]  # physical buffer indices

# 2) Sort bases so oldest comes first (chronological order)
ages      = (bases - ptr) % N        # 0 = oldest, up to (#valid-1)=newest
sort_idx  = ages.argsort()
bases     = bases[sort_idx]

# 3) Build sequence indices and gather
seq_idx    = (bases.unsqueeze(1) - offsets) % N  # (batch, seq_len)
#chron  = (bases - ptr) % N
#bases  = bases[chron.argsort()]

print("chronological bases ->", bases.tolist())
seq_idx = (bases.unsqueeze(1) - offsets) % N
for b,row in zip(bases.tolist(), seq_idx.tolist()):
    print(f"base {b}: {row}")

chronological bases -> [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
base 53: [53, 52, 50, 47, 43, 38, 32]
base 54: [54, 53, 51, 48, 44, 39, 33]
base 55: [55, 54, 52, 49, 45, 40, 34]
base 56: [56, 55, 53, 50, 46, 41, 35]
base 57: [57, 56, 54, 51, 47, 42, 36]
base 58: [58, 57, 55, 52, 48, 43, 37]
base 59: [59, 58, 56, 53, 49, 44, 38]
base 60: [60, 59, 57, 54, 50, 45, 39]
base 61: [61, 60, 58, 55, 51, 46, 40]
base 62: [62, 61, 59, 56, 52, 47, 41]
base 63: [63, 62, 60, 57, 53, 48, 42]
base 64: [64, 63, 61, 58, 54, 49, 43]
base 65: [65, 64, 62, 59, 55, 50, 44]
base 66: [66, 65, 63, 60, 56, 51, 45]
base 67: [67, 66, 64, 61, 57, 52, 46]
base 68: [68, 67, 65, 62, 58, 53, 47]
base 69: [69, 68, 66, 63, 59, 54, 48]
base 70: [70, 69, 67, 64

In [107]:
-1%30

29

Testing 